In [ ]:
library("limma")
library("dplyr")
library("tidyverse")
library("caret")
library("switchBox")

# 1、Data Processing

**Steps:**
1. Data preprocessing
2. Split training, testing, and external independent validation sets

### 1.1 Data preprocessing

In [5]:
# Define chip data processing function
array_data_process = function(data_path, expr, probeid, other_columns, gpl_file_path, genetosymbol_file_path, colnames_file_path, output_file) {
  # Read and preprocess expression data
  data = read.ilmn(data_path, expr = expr, probeid = probeid, other.columns = other_columns)
  data = neqc(data, detection.p = other_columns)
  data = data$E
  data = cbind(rownames(data), data)
  colnames(data)[1] = "ID"

  # Read gpl and genetosymbol file and filter
  GPL = read.table(gpl_file_path, header = TRUE, sep = "\t", fill = TRUE, comment.char = "#", quote = "\"")
  GPL = GPL %>% select(ID, Source, Entrez_Gene_ID) %>% filter(Source == "RefSeq") %>% select(ID, Entrez_Gene_ID)
  genetosymbol = read.table(genetosymbol_file_path, header = FALSE, sep = "\t", fill = TRUE, quote = "\"")
  genetosymbol = genetosymbol[,c(2,3)]
  colnames(genetosymbol) = c("Entrez_Gene_ID", "Symbol")
    
  # Remove duplicate gene symbols
  data = merge(GPL, data, by = "ID")
  data = merge(genetosymbol, data, by = "Entrez_Gene_ID")
  data = data[, -c(1, 3)]
  data[,-1] = apply(data[,-1], 2, as.numeric)
  index = order(rowMeans(data[,-1]), decreasing = TRUE)
  data = data[index, ]
  keep = !duplicated(data$Symbol)
  data = data[keep, ]
  rownames(data) = data[, 1]
  data = as.matrix(data[, -1])

  # Rename column names
  Colnames = read.table(colnames_file_path, header = FALSE, sep = "\t")
  match_result = match(Colnames[, 1], colnames(data))
  data = data[, match_result]
  colnames(data) = Colnames[, 2]

  # Save processed data
  write.table(data, output_file, sep = "\t")
}

In [6]:
# Function call
data_path = "./data/01_Raw_data/GSE19444/GSE19444_non-normalized.txt"
expr = "VALUE_"
probeid = "ID_REF"
other_columns = "Detection PVal"
gpl_file_path = "./data/01_Raw_data/GSE19439/GPL6947-13512.txt"
genetosymbol_file_path = "./data/01_Raw_data/GSE19439/genetosymbol.txt"
colnames_file_path = "./data/01_Raw_data/GSE19444/Colnames.txt"
output_file = "./data/02_Preliminary_data/GSE19444/data.txt"
array_data_process(data_path, expr, probeid, other_columns, gpl_file_path, genetosymbol_file_path, colnames_file_path, output_file)

Reading file ./data/01_Raw_data/GSE19444/GSE19444_non-normalized.txt ... ...


Note: inferring mean and variance of negative control probe intensities from the detection p-values.



### 1.2 Split training, testing, and external independent validation sets

In [2]:
# Merge data
base_path = "./data/02_Preliminary_data/"
folder_list = list.files(base_path, full.names = TRUE)
data = list()
for (folder in folder_list) {
  folder_name = basename(folder)
  file_path = file.path(folder, "data.txt")
  if (file.exists(file_path)) {
    data_tmp = read.table(file_path, header = TRUE, sep = "\t", check.names = FALSE)
    data_tmp = cbind(rownames(data_tmp), data_tmp)
    colnames(data_tmp)[1] = "Symbol"
    data[[folder_name]] = data_tmp
    print(paste("Data loaded with name:", folder_name))
  } else {
    cat("File not found:", folder_name, "\n")
  }
}    
all_data = Reduce(function(x, y) merge(x, y, by = "Symbol"), data)
rownames(all_data) = all_data[,1]
all_data = all_data[,-1]
all_data

[1] "Data loaded with name: GSE19439"
[1] "Data loaded with name: GSE19442"
[1] "Data loaded with name: GSE19444"
[1] "Data loaded with name: GSE83456"


In [4]:
# Split training and testing sets
selected_cols = grep("^(01|02|03)", colnames(all_data))  
Train_Test_data = all_data[, selected_cols]
Train_Test_data = t(Train_Test_data)
label = ifelse(grepl("_PTB_", rownames(Train_Test_data)), 1, 0)
Train_Test_data = as.data.frame(cbind(Train_Test_data, label = label))
set.seed(123)  
split_index = createDataPartition(Train_Test_data$label, p = 0.7, list = FALSE)
train_data = Train_Test_data[split_index, ]
test_data = Train_Test_data[-split_index, ]
write.csv(train_data, "./data/03_Final_data/01_train_data.csv")
write.csv(test_data, "./data/03_Final_data/02_test_data.csv")

# Split external independent validation set
selected_cols = grep("^(04)", colnames(all_data))  
valid_data = all_data[, selected_cols]
valid_data = t(valid_data)
label = ifelse(grepl("_PTB_", rownames(valid_data)), 1, 0)
valid_data = as.data.frame(cbind(valid_data, label = label))
write.csv(valid_data, "./data/03_Final_data/03_valid_data.csv")

# 2、Gene pair methods based on gene expression values

### 2.1 GERs

#### 2.1.1 Train

In [5]:
# Define differential analysis function
performDEGAnalysis = function(filepath, prefixes, classifyString, adjPValThreshold, logFCThreshold, Samples, limma_results_filepath, DEG_results_filepath) {
  # Read and process data
  DEGAnalysis_data = read.csv(filepath, check.names = FALSE, row.names = 1)
  DEGAnalysis_data = DEGAnalysis_data[, -ncol(DEGAnalysis_data)]
  DEGAnalysis_data = t(DEGAnalysis_data)
  DEGAnalysis_data = log2(DEGAnalysis_data + 1)
    
  # Initialize two empty data frames to store all results
  limma_results = data.frame()
  DEG_results = data.frame(Data = character(), Gene = character(), adjust_P_val = numeric(), logFC = numeric(), stringsAsFactors = FALSE)
  
  i = 1
  for(prefix in prefixes) {
    print(prefix)

    # Extract data
    col_names = grep(paste0("^", prefix), colnames(DEGAnalysis_data), value = TRUE)
    data = DEGAnalysis_data[, col_names]
    data = data[which(rowSums(data) != 0), ]

    # Change order
    pos_columns = grep(classifyString, colnames(data), value = TRUE)
    neg_columns = setdiff(colnames(data), pos_columns)
    new_column_order = c(pos_columns, neg_columns)
    data = data[, new_column_order]
      
    # Create group list
    total_columns = dim(data)[2]
    count_contains = sum(grepl(classifyString, colnames(data)))
    count_not_contains = total_columns - count_contains
    group_list = c(rep('Pos', count_contains), rep('Neg', count_not_contains))
    
    # Create design matrix
    design = model.matrix(~0 + factor(group_list))
    colnames(design) = levels(factor(group_list))
    rownames(design) = colnames(data)
    
    # Perform differential expression analysis
    fit = lmFit(data, design)
    contrast.matrix = makeContrasts(Pos-Neg, levels = design)
    fit2 = contrasts.fit(fit, contrast.matrix)
    fit2 = eBayes(fit2)
    
    # Extract differentially expressed genes
    DEG = topTable(fit2, n = Inf)
    DEG = na.omit(DEG)
    DEG_filter = DEG[DEG$adj.P.Val < adjPValThreshold & abs(DEG$logFC) > logFCThreshold, ]
    
    # Construct current analysis results data frame and append 
    limma_results = rbind(limma_results, DEG)
    tempResults = data.frame(Sample = rep(Samples[i], nrow(DEG_filter)), Gene = row.names(DEG_filter), adjust_P_val = DEG_filter$adj.P.Val, logFC = DEG_filter$logFC, stringsAsFactors = FALSE)
    DEG_results = rbind(DEG_results, tempResults)
    i = i + 1
  }
    
  # Output results
  write.csv(limma_results, limma_results_filepath, row.names = TRUE)
  write.csv(DEG_results, DEG_results_filepath, row.names = FALSE)
  return(list(limma_results = limma_results, DEG_results = DEG_results))
}

In [10]:
# Call function to perform differential analysis
filepath = "./data/03_Final_data/01_train_data.csv"
prefixes = c("0")
classifyString = "_PTB_"
adjPValThreshold = 0.0000000015
logFCThreshold = 0
Samples = c("train_data")
limma_results_filepath = "./result/01_GERs/01_limma_results.csv"
DEG_results_filepath = "./result/01_GERs/02_limma_DEG_results.csv"
Results = performDEGAnalysis(filepath, prefixes, classifyString, adjPValThreshold, logFCThreshold, Samples, limma_results_filepath, DEG_results_filepath)

[1] "0"


In [11]:
# Save ratio pairs
upregulated = Results$DEG_results$Gene[Results$DEG_results$logFC > 0]
downregulated = Results$DEG_results$Gene[Results$DEG_results$logFC < 0]
result = expand.grid(Upregulated = upregulated, Downregulated = downregulated)
write.csv(result, "./result/01_GERs/03_ratio_pair.csv")

#### 2.1.2 Test

In [2]:
# Define accuracy calculation function
calculate_accuracy = function(df, ratio_pair) {
  # Extract labels
  labels = df[, ncol(df)]
  df = df[, -ncol(df)]
  
  # Get number of samples and combinations
  num_samples = nrow(df)
  num_pairs = nrow(ratio_pair)
  
  # Store prediction results
  predictions = numeric(num_samples)
  
  # Iterate through samples
  for (i in 1:num_samples) {
    sample_data = df[i, ]
    count_positive = 0
    
    for (j in 1:num_pairs) {
      gene1 = ratio_pair[j, 1]
      gene2 = ratio_pair[j, 2]

      if (sample_data[gene1] > sample_data[gene2]) {
        count_positive = count_positive + 1
      }
    }
    
    # Determine if positive class based on condition
    if (count_positive > num_pairs / 2) {
      predictions[i] = 1  
    } else {
      predictions[i] = 0  
    }
  }
  
  # Calculate accuracy
  accuracy = sum(predictions == labels) / num_samples
  
  # Output accuracy
  return(accuracy)
}

In [4]:
# Call function to calculate accuracy
df = read.csv("./data/03_Final_data/01_train_data.csv", check.names = FALSE, row.names = 1)
ratio_pair = read.csv("./result/01_GERs/03_ratio_pair.csv", check.names = FALSE, row.names = 1)
accuracy = calculate_accuracy(df, ratio_pair)
accuracy

[1] 0.8446602

**train:0.845 ; test:0.818 ; valid:0.849**

# 3、Gene pair methods based on gene ranking relationships

### 3.1 k-TSP

#### 3.1.1 Train

In [70]:
# Import data
train_data = read.csv("./data/03_Final_data/01_train_data.csv", check.names = FALSE, row.names = 1)
train_labels = as.factor(train_data[, ncol(train_data)])
train_data = train_data[, -col(train_data)]
train_data = t(train_data)

In [71]:
# Get feature pairs
classifier = SWAP.Train.KTSP(train_data, train_labels, krange=c(1, 3, 5))
write.csv(classifier$TSPs, "./result/03_k-TSP/01_kTSP.csv", row.names = FALSE)

,gene1,gene2
"ANKRD22,KLF12",ANKRD22,KLF12
"PSTPIP2,SLC4A7",PSTPIP2,SLC4A7
"GBP4,WDR75",GBP4,WDR75
"FCGR1BP,SLAMF6",FCGR1BP,SLAMF6
"GBP6,ADAM7",GBP6,ADAM7


#### 3.1.2 Test

In [75]:
# Import data
test_data = read.csv("./data/03_Final_data/02_test_data.csv", check.names = FALSE, row.names = 1)
test_labels = as.factor(test_data[, ncol(test_data)])
test_data = test_data[, -ncol(test_data)]
test_data = t(test_data)

In [76]:
# Test
testPrediction = SWAP.KTSP.Classify(test_data, classifier)
accuracy = sum(as.vector(testPrediction) == test_labels) / length(test_labels)
accuracy

[1] 0.9090909

**train:0.942 ; test:0.909 ; valid:0.953**

### 3.2 TSPG

#### 3.2.1 Train

In [2]:
# Define function
extract_genes = function(train_data, train_labels, percentage, features_num, iterations_num, freq_threshold, output_file) {
  # Get number of samples        
  n_samples = ncol(train_data)
  n_train = floor(percentage * n_samples)
  
  # Initialize A and B lists
  A_list = list()
  B_list = list()
  
  # Repeat n times
  for (i in 1:iterations_num) {
    # Randomly sample training data
    print(i)
    flush.console()
    set.seed(i)           
    indices = sample(1:n_samples, n_train)
    data = train_data[, indices]
    labels = train_labels[indices]
    
    # Get feature gene pairs
    classifier = SWAP.Train.KTSP(data, labels, krange = features_num)
    tsp_matrix = classifier$TSPs 
    A_list[[i]] = tsp_matrix[, 1] 
    B_list[[i]] = tsp_matrix[, 2] 
  }
  
  # Merge A and B lists
  A_genes = unlist(A_list)
  B_genes = unlist(B_list)
  
  # Calculate frequencies
  A_freq = table(A_genes) / iterations_num
  B_freq = table(B_genes) / iterations_num
  
  # Keep genes with frequencies above the threshold
  A_selected = names(A_freq[A_freq > freq_threshold])
  B_selected = names(B_freq[B_freq > freq_threshold])
  
  # Create result matrix
  result_matrix = cbind(A = A_selected, B = B_selected)
  
  # Save matrix as file
  write.csv(result_matrix, file = output_file)
  
  return(result_matrix)
}

In [3]:
# Call function
train_data = read.csv("./data/03_Final_data/01_train_data.csv", check.names = FALSE, row.names = 1)
train_labels = as.factor(train_data[, ncol(train_data)])
train_data = train_data[, -ncol(train_data)]
train_data = t(train_data)
percentage = 0.9
features_num = 20
iterations_num = 1000
freq_threshold = 0.8
output_file = "./result/04_TSPG/01_selected_genes.csv"
extract_genes(train_data, train_labels, percentage, features_num, iterations_num, freq_threshold, output_file)

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

A,B
ANKRD22,CA5B
CEACAM1,FCMR
FCGR1A,GZMK
FCGR1BP,SIRPG
GBP1,SLAMF6
GBP4,SS18L2
GBP5,CA5B
GBP6,FCMR
LHFPL2,GZMK
PSMB9,SIRPG


#### 3.2.2 Test

In [11]:
# Define function
rank_based_classifier = function(data, gene_matrix, labels) {
  # Extract relevant gene matrix
  all_genes = c(gene_matrix[, 1], gene_matrix[, 2])
  data_subset = data[all_genes, , drop = FALSE]
  
  # Initialize prediction result vector
  predictions = numeric(ncol(data))
  
  # Iterate through each sample
  for (i in 1:ncol(data)) {
    print(i)
    flush.console()
      
    # Extract gene data for each sample
    sample_data = data_subset[, i]
    
    # Apply rank transformation
    ranked_data = rank(sample_data)
    
    # Get average rank value
    A_genes = gene_matrix[, 1]
    B_genes = gene_matrix[, 2]
    A_ranks = ranked_data[A_genes]
    B_ranks = ranked_data[B_genes]
    A_mean_rank = mean(A_ranks, na.rm = TRUE)
    B_mean_rank = mean(B_ranks, na.rm = TRUE)
    
    # Make prediction
    if (A_mean_rank < B_mean_rank) {
      predictions[i] = 0
    } else {
      predictions[i] = 1
    }
  }

  accuracy <- sum(predictions == labels) / length(labels)
  return(accuracy)
}

In [12]:
# Call function
data = read.csv("./data/03_Final_data/03_valid_data.csv", check.names = FALSE, row.names = 1)
labels = as.factor(data[, ncol(data)])
data = data[, -ncol(data)]
data = t(data)
gene_matrix = read.csv("./result/04_TSPG/01_selected_genes.csv", check.names = FALSE, row.names = 1)
rank_based_classifier(data, gene_matrix, labels)

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106


[1] 0.9245283

**train:0.913 ; test:0.886 ; valid:0.925**

# 4、Plot

In [31]:
# Define function
create_polar_bar_chart = function(input_file, y_limit, y_breaks, colors, output_file) {
  # Read data
  data = read.csv(input_file, sep = ',', header = TRUE, stringsAsFactors = FALSE)
    
  # Add order column
  data = data %>% mutate(Order = 1:n()) # Add a column for specifying order
  data$Order = factor(data$Order) # Convert to factor type for color assign
  
  # Plot polar bar chart
  p = data %>% ggplot(aes(x = Order, y = ACC, fill = Order)) + 
    geom_bar(stat = 'identity') + # Plot bars with height using raw values, no additional aggregation
    coord_polar(theta = "y") + # Apply polar transformation
    theme_bw() + # Remove background
    theme_minimal() + # Remove borders
    xlab("") + ylab("") + # Remove x and y axis labels
    # Remove legend, y-axis ticks, and minor grid lines
    theme(legend.position = "none", axis.text.y = element_blank(), panel.grid.minor = element_blank()) + 
    # Add labels: content, position, font, size, alignment
    geom_text(aes(label = Method, x = Order, y = 0, fontface = "italic"), size = 3, hjust = 1) +
    # Set range ticks
    scale_y_continuous(limits = c(0, y_limit), breaks = seq(0, y_limit, by = y_breaks)) + 
    scale_fill_manual(values = colors) # Set fill color
  
  # Save as PDF
  ggsave(output_file, plot = p, height = 6, width = 6, dpi = 300)
}

In [35]:
# Run
input_file = './result/09_plot/valid_ACC.csv'
y_limit = 1
y_breaks = 0.1
colors = c("#ffef1e", "#fdfa0a", "#deff02", "#c9ff00", "#b7ff02", "#82e0aa", "#7dcea0", "#73c6b6", "#76d7c4")
output_file = './result/09_plot/03_valid_ACC.pdf'
create_polar_bar_chart(input_file, y_limit, y_breaks, colors, output_file)